# Scrapping

*Thomas VENDRAMINI*

In [85]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd


def scrapper(mot):
    session = HTMLSession()
    response = session.get("https://dictionary.cambridge.org/fr/dictionnaire/anglais/"+mot+"")
    audio = response.content
    soup = BeautifulSoup(audio,'html.parser')
    finded = soup.findAll('div', {"class" : "pos-header dpos-h"})
    path = 'https://dictionary.cambridge.org'
    liste_langue = {'us', 'uk'}
    df = pd.DataFrame(columns=['mot','langue','type','prononciation','url'])
    for langue in liste_langue :
        for audio in finded :
            try :
                mot = audio.find('div',{"class":"di-title"}).text
                dtype = audio.find("div",{"class":"posgram dpos-g hdib lmr-5"}).text
                langue_recup = audio.find("span",{"class":langue +" dpron-i"})
                prononciation = audio.find("span",{"class":"pron dpron"}).text          
                url = path+langue_recup.find('source',{"type":"audio/mpeg"})['src']           
                new_row = {'mot':mot, 'langue':langue,'type':dtype,'prononciation':prononciation,'url':url}
                df = df.append(new_row, ignore_index=True)
            except :
                pass
                #print('langue: {}'.format(langue))
                #print('type : {}'.format(dtype))
                #print('prononciation : {}'.format(prononciation))
                #print('url : {}'.format(url))
                #print("")
    df['type'] = df['type'].str.replace(' \[ \w \]', '')
    df = df.drop_duplicates(['langue'])
    uk = pd.DataFrame(df.reset_index(drop= True).loc[0,['mot','type','prononciation', 'url']]).T
    uk = uk.rename(columns={"prononciation":"prononciation_uk","url":"url_uk"})
    us = pd.DataFrame(df.reset_index(drop= True).loc[1,['mot','prononciation', 'url']]).T
    us = us.rename(columns={"prononciation":"prononciation_us","url":"url_us"})
    df_retour = uk.merge(us, on='mot', suffixes=[None,None])
    return df_retour

In [86]:
scrapper("bed")

,mot,type,prononciation_uk,url_uk,prononciation_us,url_us
0,bed,noun,/bed/,https://dictionary.cambridge.org/fr/media/angl...,/bed/,https://dictionary.cambridge.org/fr/media/angl...
